<br>

# _Imports_ e Funções


In [ ]:
import os
import shutil
import zipfile
from pathlib import Path

import geopandas as gpd
import requests
from paths import input_path, output_path

<br>

# Dados Espaciais


## Limites Municipais


In [ ]:
# Define qual o código IBGE do Estado
estado = 'SP'

# Define o nome do arquivo que será salvo as informações do IBGE
file = input_path / f'{estado}.zip'

# URL para baixar o arquivo zip
url = f'http://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2018/UFs/{estado}/{estado.lower()}_municipios.zip'

# Print
url

In [ ]:
filename = Path(url).name
filename

In [ ]:
# Download
r = requests.get(url, allow_redirects=True)
open(input_path / filename, 'wb').write(r.content)

In [ ]:
# Unzip
input_file = input_path / filename
input_file

input_path_temp = input_path / 'temp'
input_path_temp.mkdir(exist_ok=True)


In [ ]:
with zipfile.ZipFile(input_file, 'r') as zip_ref:
    zip_ref.extractall(input_path_temp)

In [ ]:
# Lista Arquivos
#os.listdir(temp)
list(input_path_temp.glob('*'))

In [ ]:
# Read shapefile
gdf = gpd.read_file(input_path_temp  /'35MUE250GC_SIR.shp')
display(gdf.head(5))
gdf.plot()

In [ ]:
gdf = gdf[['CD_GEOCMU', 'NM_MUNICIP', 'geometry']]

# Renomeia Colunas
gdf = gdf.rename(
    columns={'NM_MUNICIP': 'nome_municipio', 'CD_GEOCMU': 'id_municipio'}
)

# Results
print(gdf.dtypes)
display(gdf.head(5))

In [ ]:
# Reprojeta
print(gdf.crs)
gdf = gdf.to_crs(epsg=4326)
print(gdf.crs)
gdf.plot()

In [ ]:
# Salva
gdf.to_file(
    output_path / 'limite_municipal_sp.geojson',
    driver='GeoJSON',
    encoding='utf-8',
)
gdf.to_file(
    output_path / 'limite_municipal_sp.gpkg',
    layer='Limite',
    driver='GPKG',
)

In [ ]:
# Excluí pasta temporária
shutil.rmtree(input_path_temp)